In [2]:
import numpy as np
import json

In [120]:
with open("experiments/dataset_infos.json", "r") as json_file:
    datasets = json.load(json_file)

In [121]:
# data_list = datasets
## Create a new dictionary with the desired structure
# dataset_dict = {}
# for data_dict in data_list:
#    if 'dataset' in data_dict:
#        dataset_name = data_dict['dataset']
#        del data_dict['dataset']
#        dataset_dict[dataset_name] = data_dict

In [122]:
# with open("dataset_infos.json", "w") as json_file:
#    json.dump(dataset_dict, json_file)

In [123]:
import os

list_datasets = []
for dataset in os.listdir("datasets/Classical"):
    list_datasets.append(dataset.replace(".npz", ""))

In [124]:
def generate_dictionaries_with_dependency(input_dict, dependency_rule):
    import itertools

    # Separate the keys and values, and prepare for combinations excluding the dependent key
    keys, values = zip(
        *((k, v) for k, v in input_dict.items())
    )  # if k != "latent_dim"))
    combinations = list(itertools.product(*values))

    # Create dictionaries with combinations of values and add the dependent key-value pair
    result_dicts = []
    for combination in combinations:
        new_dict = dict(zip(keys, combination))
        #    # Apply the dependency rule to determine the value of the dependent key
        #    new_dict["latent_dim"] = dependency_rule(new_dict["dataset"])
        result_dicts.append(new_dict)

    return result_dicts

In [125]:
# Define a rule for the dependency of 'key4' on the value of 'key1'
import math


def dependency_rule(value_of_key1: str):
    dataset = value_of_key1.split("_")[1]
    n_features = datasets[dataset]["n_features"]
    if n_features > 30:
        latent_dim = 15
    elif n_features >= 4 and n_features <= 30:
        latent_dim = n_features / 2
    else:
        latent_dim = 2
    return int(latent_dim)

In [126]:
predefined_lists_with_dependency = {
    "dataset": list_datasets,
    "kernel": ["rbf"],
    "batch_size": ["128"],
    "learning_rate": ["0.01"],
    # "latent_dim": ["2","5"],
    "loss": ["normal"],  # "loe_hard", "loe_soft"],
    "anomaly_type": ["normal"],  # ,"global", "local", "dependency", "cluster"],
    "noise_type": ["normal"],  # ,"irrelevant_features", "duplicated_anomalies"],
    "noise_ratio": ["0"],  # "0.01","0.05","0.1","0.25","0.50",],
    "layers": ["5,5", "10,10", "15,15", "10,10,10"],
    "n_inducing": ["25", "50", "100"],
    "n_epochs": ["1000"],
}

In [127]:
experiments = generate_dictionaries_with_dependency(
    predefined_lists_with_dependency, dependency_rule
)

In [130]:
for experiment in experiments:
    dataset_name = experiment["dataset"].split("_")[1]
    experiment.update(datasets[dataset_name])

In [132]:
with open("experiments/refine/001_normal_study.json", "w") as json_file:
    json.dump(experiments, json_file)

In [150]:
for experiment in experiments:
    if experiment["latent_dim"] >=4:
        experiment["latent_dim"] = int(experiment["latent_dim"]/2)
    elif experiment["latent_dim"] == 3:
        experiment["latent_dim"] = 2
    else:
        experiment["latent_dim"] = 1

In [152]:
with open("experiments/refine/001_normal_study_latent_quarter.json", "w") as json_file:
    json.dump(experiments, json_file)

In [151]:
pd.DataFrame(experiments).sort_values("latent_dim")["latent_dim"].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [153]:
with open("experiments/refine/001_normal_study_latent_half.json") as json_file:
    exp_half = json.load(json_file)
with open("experiments/refine/001_normal_study_latent_quarter.json") as json_file:
    exp_quarter = json.load(json_file)

In [157]:
experiments = exp_half + exp_quarter

In [165]:
df = pd.DataFrame(experiments).sort_values(by = ["dataset", "latent_dim"] ).reset_index(drop=True)

In [179]:
df = df[['dataset', 'n_samples', 'n_features', 'n_anomaly', 'pct_anomaly',
       'domain','kernel', 'batch_size', 'learning_rate','loss',
       'anomaly_type', 'noise_type', 'noise_ratio', 'latent_dim', 'layers', 'n_inducing',
       'n_epochs',]]

In [181]:
df.to_json("experiments/refine/001_complete_normal_study.json", orient = 'records')